## Create A QnA Knowledge Base

1. Create an Azure QnA Maker resource in the Azure portal.

In [49]:
import http.client, urllib.request, urllib.parse, urllib.error, base64, json, urllib

qna_endpoint = "YOUR_QNA_SERVICE.cognitiveservices.azure.com"
qna_key = "YOUR_QNA_KEY"
faq_url = "https://aka.ms/kmoh-faq"

headers = {
    'Content-Type': 'application/json',
    'Ocp-Apim-Subscription-Key': qna_key
}

body = {
    "name": "Margies FAQ",
    "files": [
    {
      "fileName": "Margies Travel FAQ.docx",
      "fileUri": faq_url
    }
  ]
}



# Execute the REST API call and get the response.
conn = http.client.HTTPSConnection(qna_endpoint)
request_path = "/qnamaker/v4.0/knowledgebases/create"
conn.request("POST", request_path, json.dumps(body), headers)
response = conn.getresponse()
print(response.status)
data = response.read().decode("UTF-8")
result = None
if len(data) > 0:
    result = json.loads(data)
    print(json.dumps(result, sort_keys=True, indent=2))



202
{
  "createdTimestamp": "2019-08-20T05:22:57Z",
  "lastActionTimestamp": "2019-08-20T05:22:57Z",
  "operationId": "74248892-9b23-422e-9605-e635f35edd22",
  "operationState": "NotStarted",
  "userId": "c6dceea61f344163b1e277026c98e8af"
}


In [51]:
# Keep running until succeeded

operation_id = result["operationId"]
request_path = "/qnamaker/v4.0/operations/" + operation_id
conn2 = http.client.HTTPSConnection(qna_endpoint)
conn2.request("GET", request_path, None, headers)
response = conn2.getresponse()
print(response.status)
data = response.read().decode("UTF-8")
result = None
if len(data) > 0:
    result = json.loads(data)
    print(json.dumps(result, sort_keys=True, indent=2))

200
{
  "createdTimestamp": "2019-08-20T05:22:57Z",
  "lastActionTimestamp": "2019-08-20T05:23:14Z",
  "operationId": "74248892-9b23-422e-9605-e635f35edd22",
  "operationState": "Succeeded",
  "resourceLocation": "/knowledgebases/8282a6a3-9eb3-4439-9f37-0f938aee54de",
  "userId": "c6dceea61f344163b1e277026c98e8af"
}


In [52]:
# Publish the KB
qna_kb = result['resourceLocation']
request_path = "/qnamaker/v4.0" + qna_kb
conn3 = http.client.HTTPSConnection(qna_endpoint)
conn3.request("POST", request_path, None, headers)
response = conn3.getresponse()
print(response.status)
data = response.read().decode("UTF-8")
result = None
if len(data) > 0:
    result = json.loads(data)
    print(json.dumps(result, sort_keys=True, indent=2))

204


In [53]:
# Get the endpoint key
request_path = "/qnamaker/v4.0/endpointkeys"
conn4 = http.client.HTTPSConnection(qna_endpoint)
conn4.request("GET", request_path, None, headers)
response = conn4.getresponse()
print(response.status)
data = response.read().decode("UTF-8")
result = None
if len(data) > 0:
    result = json.loads(data)
    print(json.dumps(result, sort_keys=True, indent=2))
    endpoint_key = result["primaryEndpointKey"]

200
{
  "installedVersion": "5.15.0",
  "lastStableVersion": "5.15.0",
  "primaryEndpointKey": "6cca4cd6-2907-4897-89e7-c5f64751bb76",
  "secondaryEndpointKey": "d7909396-d6b9-42df-95d4-7dd55b95dc7c"
}


In [54]:
def submit_question(question):
    import http.client, urllib.parse, json, time, sys

    qna_host = "margies-qna.azurewebsites.net"
    qna_route = "/qnamaker" + qna_kb + "/generateAnswer"

    headers = {
                'Authorization': 'EndpointKey ' + endpoint_key,
                'Content-Type': 'application/json'
              }

    body = {'question': question}
    
    answer = ""
    
    try:
        conn4 = http.client.HTTPSConnection(qna_host,port=443)
        conn4.request ("POST", qna_route,  str(body), headers)
        response = conn4.getresponse()
        response_content = response.read()
        response_json = json.loads(response_content)
        #print(json.dumps(response_json))
        answer = response_json["answers"][0]["answer"]
    except Exception as ex:
        answer = ex
        
    return answer

question = input("Enter a question for Margies Travel\n\n")
answer = submit_question(question)
print("\n%s" % answer)


Enter a question for Margies Travel

How can I cancel a reservation?

**Canceling a Flight or Hotel**

You can cancel a flight or hotel anytime up to 24 hours before check-in. A cancellation fee may apply. Call us on 555 123 456; or visit our website: [www.margiestravel.com ](http://www.margiestravel.com/).
